## Script to convert BMP kodak dataset images to PNG image for later comparison

In [2]:
import os
from PIL import Image
from pathlib import Path
import glob
import subprocess

current_path = Path('.').resolve()
print(str(current_path))

/home/filippo/DataspellProjects/ImageCompressionAI


In [53]:
source_path = str(current_path)+'/IMAGES/BMP_IMAGES'
print("Source: ",source_path)
destination_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/BMP_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES


In [54]:
for file in glob.glob(str(source_path)+'/*.bmp'):
    image = Image.open(file)
    image = image.convert('RGB')
    print(Path(file).stem)
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    image.close()

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Script to convert PNG images in YUV lossless for VVC encoding
In order to perform the conversion a working installation of FFmpeg is needed


In [48]:
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Source: ",source_path)
destination_path = str(current_path)+'/IMAGES/YUV_IMAGES'
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES


In [49]:
for file in os.listdir(source_path):
    image = Image.open(source_path+'/'+file)
    with open(str(destination_path)+'/'+str(Path(file).stem)+'.txt', 'w') as f:
        f.write(str(image.width)+'x'+str(image.height))
    source_file = str(source_path) + '/' + str(file)
    destination_file = str(destination_path) + '/' + str(Path(file).stem) + '.yuv'
    subprocess.run(['ffmpeg','-y','-i',str(source_file),'-f','rawvideo','-vf','scale=in_range=full:out_range=full,format=yuv420p10le',str(destination_file)],stdout = subprocess.DEVNULL)
    print(Path(file).stem)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id=sha1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb -

IMG0001


Output #0, rawvideo, to '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0002.yuv':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0: Video: rawvideo (Y3[11][10] / 0xA0B3359), yuv420p10le(pc, progressive), 3072x2048, q=2-31, 2359296 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc60.3.100 rawvideo
frame=    1 fps=0.0 q=-0.0 Lsize=   18432kB time=00:00:00.00 bitrate=N/A speed=   0x    eed=N/A    
video:18432kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -W

IMG0002


Output #0, rawvideo, to '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0003.yuv':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0: Video: rawvideo (Y3[11][10] / 0xA0B3359), yuv420p10le(pc, progressive), 3072x2048, q=2-31, 2359296 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc60.3.100 rawvideo
frame=    1 fps=0.0 q=-0.0 Lsize=   18432kB time=00:00:00.00 bitrate=N/A speed=   0x    eed=N/A    
video:18432kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -W

...

Output #0, rawvideo, to '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0022.yuv':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0: Video: rawvideo (Y3[11][10] / 0xA0B3359), yuv420p10le(pc, progressive), 3072x2048, q=2-31, 2359296 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc60.3.100 rawvideo
frame=    1 fps=0.0 q=-0.0 Lsize=   18432kB time=00:00:00.00 bitrate=N/A speed=   0x    eed=N/A    
video:18432kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -W

IMG0022


Output #0, rawvideo, to '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0023.yuv':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0: Video: rawvideo (Y3[11][10] / 0xA0B3359), yuv420p10le(pc, progressive), 3072x2048, q=2-31, 2359296 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc60.3.100 rawvideo
frame=    1 fps=0.0 q=-0.0 Lsize=   18432kB time=00:00:00.00 bitrate=N/A speed=   0x    eed=N/A    
video:18432kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -W

IMG0023
IMG0024


Output #0, rawvideo, to '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0024.yuv':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0: Video: rawvideo (Y3[11][10] / 0xA0B3359), yuv420p10le(pc, progressive), 3072x2048, q=2-31, 2359296 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc60.3.100 rawvideo
frame=    1 fps=0.0 q=-0.0 Lsize=   18432kB time=00:00:00.00 bitrate=N/A speed=   0x    eed=N/A    
video:18432kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000000%


# Script to convert YUV images back into PNG

In [3]:
dimensions_path = str(current_path)+'/IMAGES/YUV_IMAGES'
print('Dimensions: ',dimensions_path)
source_path = str(current_path)+'/IMAGES/VVC_IMAGES/YUV'
print("Source: ",source_path)
destination_path = str(current_path)+'/IMAGES/VVC_IMAGES'
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    print("Destination: ",destination_path)
else:
    os.mkdir(destination_path)

Dimensions:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES
Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/YUV
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES


In [4]:
for file in os.listdir(source_path):
    if file.lower().endswith('.yuv'):
        dimensions = open(str(dimensions_path)+'/'+str(Path(file).stem)+'.txt').readline()
        source_file = str(source_path) + '/' + str(file)
        destination_file = str(destination_path) + '/' + str(Path(file).stem) + '.png'
        subprocess.run(['ffmpeg','-y','-s',str(dimensions),'-pix_fmt','yuv420p10le','-i',str(source_file),'-f','image2','-vframes','1','-pix_fmt','rgb24',str(destination_file)],stdout = subprocess.DEVNULL)
        print(Path(file).stem)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id=sha1 ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb -

IMG0001


[image2 @ 0x56305b511e40] The specified filename '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/IMG0002.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x56305b511e40] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.00 bitrate=N/A speed=   0x    
video:8465kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASS

IMG0002


[image2 @ 0x556da22ede40] The specified filename '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/IMG0003.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x556da22ede40] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.00 bitrate=N/A speed=   0x    
video:4733kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASS

...

[image2 @ 0x55d7cb921e40] The specified filename '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/IMG0022.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x55d7cb921e40] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.00 bitrate=N/A speed=   0x    
video:10051kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_AS

IMG0022


[image2 @ 0x55bb55476e40] The specified filename '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/IMG0023.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x55bb55476e40] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.00 bitrate=N/A speed=   0x    
video:6483kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASS

IMG0023
IMG0024


[image2 @ 0x55902ce0ae40] The specified filename '/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/IMG0024.png' does not contain an image sequence pattern or a pattern is invalid.
[image2 @ 0x55902ce0ae40] Use a pattern such as %03d for an image sequence or use the -update option (with -frames:v 1 if needed) to write a single image.
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.00 bitrate=N/A speed=   0x    
video:9126kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
